In [1]:
pip install scikit-surprise==1.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 43.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp310-cp310-linux_x86_64.whl size=2051293 sha256=692d59b180c7b2e0dd3fe524bb991f331964d9b3a84c7ed912b5f62d524d9872
  Stored in directory: /root/.cache/pip/wheels/14/e3/9f/653d3704a5542efd9637961ecce07b6c42008ca912881c1828
Successfully built scikit-surprise


In [2]:
from surprise import accuracy, Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [3]:
import pandas as pd


In [6]:
dataset = 'ratings_small.csv'
# dataset.exists()

In [7]:
df = pd.read_csv(dataset)
df['rating'].dropna(inplace=True)
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
df.rating.max(), df.rating.min()

(5.0, 0.5)

In [9]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [10]:
algo = SVD(verbose=True, n_epochs=20)
cross_validate(algo, data, measures=['RMSE', "MAE"], cv=4, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing

{'test_rmse': array([0.89717023, 0.88860908, 0.89977818, 0.91150704]),
 'test_mae': array([0.69028413, 0.68523611, 0.69301086, 0.7037954 ]),
 'fit_time': (6.714494943618774,
  5.469404220581055,
  6.7532477378845215,
  5.405574798583984),
 'test_time': (0.27810096740722656,
  0.14152860641479492,
  0.2378096580505371,
  0.2824673652648926)}

In [11]:
trainset = data.build_full_trainset()
algo.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [12]:
testset = trainset.build_testset()
predictions = algo.test(testset)
# RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True)
# accuracy.mae(predictions, verbose=True)

RMSE: 0.6442


0.644179421930242

In [13]:
sample_row = df.sample(n=1)
userId = sample_row['userId'].values[0]
movieId = sample_row['movieId'].values[0]
print(userId, movieId)

85 454


In [14]:
pred = algo.predict(uid=userId, iid=movieId)
pred.est

3.3038332072919276

In [15]:
import pickle

In [16]:
algo_data = {"model": algo}
with open('model.pkl', 'wb') as f:
    pickle.dump(algo_data, f)

In [17]:
model_algo= None
with open('model.pkl', 'rb') as f:
    model_data_loaded = pickle.load(f)
    model_algo = model_data_loaded.get('model')

In [18]:
model_algo.predict(uid=userId, iid=movieId).est

3.3038332072919276

In [19]:
sample_rows = df.sample(n=10).to_dict('records')
for row in sample_rows:
    userId = row['userId']
    movieId = row['movieId']
    pred = model_algo.predict(uid=userId, iid=movieId).est
    print(userId, movieId, pred)

195 1104 3.373960465002153
500 4701 2.8090604020501146
195 1964 2.7622073764348753
452 5010 3.3948685982622355
358 104 1.509784250015907
380 8369 2.8184370994231682
460 1196 4.0970448769271
468 2612 3.428936291313589
468 3108 3.0780560595151303
98 4878 4.48992416811832
